# Purpose
(Step 3.) Filter data to focus on relevant subset

In [5]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

In [6]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data" 
RAW_DATA_DIR = DATA_DIR / "raw"
CSV_DATA_DIR = DATA_DIR / "to_csv"
FILTERED_DATA_DIR = DATA_DIR / "filtered"
TXT_DATA_DIR = RAW_DATA_DIR / "20210901"
MAIN_TXT_DATA_DIR = TXT_DATA_DIR / "main"

# Look at particular MedDRA IDs related to pancreatitis
Search for pancreatitis in ... 
Nevermind, this list was sent to me.

In [66]:
meddra_id_pancreatitis = [
  "10076058",
  "10076059",
  "10033588",
  "10033626",
  "10033625",
  "10071853",
  "10055312",
  "10058096",
  "10055858",
  "10033672",
  "10019607",
  "10073794",
  "10033650",
  "10033651",
  "10056219",
  "10028891",
  "10033654",
  "10033655",
  "10054706",
  "10052400",
  "10033635",
  "10059155",
  "10076039",
  "10033645",
  "10048365",
  "10033656",
  "10033658",
  "110067190",
  "10000971",
  "10033647",
  "10033648",
  "10066715",
  "10033657",
]
len(meddra_id_pancreatitis)

33

In [67]:
# import yaml

# Force into ints since they're inferred when pulling in the ADR table

In [68]:
meddra_id_pancreatitis = [int(meddra_id) for meddra_id in meddra_id_pancreatitis]

In [7]:
table_name = "ADR"
adr = pd.read_csv(CSV_DATA_DIR / f"{table_name}.csv", index_col=0, na_values="-")
print(len(adr))
adr.head(3)

66802344


,UMCReportId,Adr_Id,MedDRA_Id,Outcome
0,34777001,15,10012578,1.0
1,34776971,27,10053762,6.0
2,34776965,5,10019211,6.0


In [70]:
adr.dtypes

UMCReportId      int64
Adr_Id           int64
MedDRA_Id        int64
Outcome        float64
dtype: object

# Filter ADR by MedDRA IDs we're interested in

In [71]:
adr = adr[adr["MedDRA_Id"].isin(meddra_id_pancreatitis)]
print(len(adr))
adr.head(3)
adr.to_csv(FILTERED_DATA_DIR / "ADR.csv")

72851


# Find unique patients with some form of pancreatitis

In [72]:
len(adr["UMCReportId"])

72851

In [73]:
len(adr["UMCReportId"].unique())

71182

In [74]:
umc_report_id_pancreatitis = adr["UMCReportId"].unique()
umc_report_id_pancreatitis

array([14226956,  4742330,  4742361, ...,  9483763,  9484075,  9484617])

# Also look at patients that followed up or changed/updated their UMC Report ID

In [75]:
table_name = "FOLLOWUP"
followup = pd.read_csv(CSV_DATA_DIR / f"{table_name}.csv", index_col=0, na_values="-")
print(table_name, "counts")
print("\tbefore:", f"{len(followup):,}")
followup = followup[followup["UMCReportId"].isin(umc_report_id_pancreatitis)]
print("\tafter:", f"{len(followup):,}")

FOLLOWUP counts
	before: 7,759,897
	after: 33,661


In [76]:
followup
new_umc_report_id_pancreatitis = followup["ReplacedUMCReportId"].unique().tolist()
# # print(len(followup["ReplacedUMCReportId"]))
# # print(len(followup["ReplacedUMCReportId"].unique()))
# umc_report_id_pancreatitis += 

In [77]:
# print(new_umc_report_id_pancreatitis)
# print(umc_report_id_pancreatitis.tolist())

umc_report_id_pancreatitis = umc_report_id_pancreatitis.tolist() + followup["ReplacedUMCReportId"].unique().tolist()

# Filter tables to only have these pancreatitis patients

In [9]:
table_names = [
  "ADR",
  "DEMO",
  "DRUG",
  "FOLLOWUP",
  # "IND",
  # "LINK",
  "OUT",
  "SRCE",
]

In [10]:
for table in table_names:
  df = pd.read_csv(CSV_DATA_DIR / f"{table}.csv", index_col=0, na_values="-")
  print(table, "counts")
  
  print("\tbefore:", f"{len(df):,}")
  df = df[df["UMCReportId"].isin(umc_report_id_pancreatitis)]
  print("\tafter:", f"{len(df):,}")
  df.to_csv(FILTERED_DATA_DIR / f"{table}.csv")

ADR counts
	before: 66,802,344


NameError: name 'umc_report_id_pancreatitis' is not defined

In [80]:
len(umc_report_id_pancreatitis)

104843

In [81]:
for table in table_names:
  df = pd.read_csv(CSV_DATA_DIR / f"{table}.csv", index_col=0, na_values="-")
  print(table, "counts")
  print("\tbefore:", f"{len(df):,}")
  df = df[df["UMCReportId"].isin(umc_report_id_pancreatitis)]
  print("\tafter:", f"{len(df):,}")
  df.to_csv(FILTERED_DATA_DIR / f"{table}.csv")

ADR counts
	before: 66,802,344
	after: 257,645
DEMO counts
	before: 27,213,386
	after: 71,182


/var/folders/xb/j24pcj0541l061x4stbhr1p40000gn/T/ipykernel_14837/917182242.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_DATA_DIR / f"{table}.csv", index_col=0, na_values="-")


DRUG counts
	before: 71,006,811
	after: 291,734
FOLLOWUP counts
	before: 7,759,897
	after: 33,661
OUT counts
	before: 22,677,257
	after: 58,674
SRCE counts
	before: 21,105,031
	after: 59,438


# Filter tables where the identifier is the drug ID, using only drugs used for the pancreatitis patients

In [82]:
table_name = "DRUG"
drug = pd.read_csv(FILTERED_DATA_DIR / f"{table_name}.csv", index_col=0, na_values="-")
drug

,UMCReportId,Drug_Id,MedicinalProd_Id,DrecNo,Seq1,Seq2,Route,Basis,Amount,AmountU,Frequency,FrequencyU
1003,14226956,1010,1510210,57100,2,002,48.0,1.0,NaN,NaN,NaN,NaN
1397,4742330,1394,6547,286,1,001,NaN,2.0,NaN,NaN,NaN,804.0
1398,4742330,1397,16828,1462,1,007,NaN,2.0,NaN,NaN,NaN,804.0
1399,4742330,1398,15060,1138,1,001,NaN,2.0,NaN,NaN,NaN,804.0
1400,4742330,1400,9014,447,1,001,NaN,2.0,NaN,NaN,NaN,804.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71006795,9484617,71006796,1361642,177,1,5,48.0,1.0,250.0,4.0,NaN,804.0
71006796,9484617,71006797,3990,162,1,1,48.0,1.0,4.0,3.0,NaN,804.0
71006797,9484617,71006798,1361631,109,2,2,42.0,1.0,1.0,2.0,NaN,804.0
71006798,9484617,71006799,2023866,17083,2,2,58.0,1.0,5.0,26.0,NaN,804.0


In [83]:
len(drug["Drug_Id"])

291734

In [84]:
drug_id_pancreatitis = drug["Drug_Id"].unique()

In [85]:
table_names = ["IND", "LINK"]
for table in table_names:
  df = pd.read_csv(CSV_DATA_DIR / f"{table}.csv", index_col=0, na_values="-")
  print(table, "counts")
  print("\tbefore:", f"{len(df):,}")
  df = df[df["Drug_Id"].isin(drug_id_pancreatitis)]
  print("\tafter:", f"{len(df):,}")
  df.to_csv(FILTERED_DATA_DIR / f"{table}.csv")

IND counts
	before: 35,474,400
	after: 127,379
LINK counts
	before: 99,273,742
	after: 507,837


In [86]:
# sns.barplot(data=adr, y="Outcome")

In [87]:
sns.barplot(data=res, x="Outcome", y="counts")

NameError: name 'res' is not defined

In [ ]:
res = adr[["Outcome"]].value_counts().reset_index()
res = res.rename({0: "counts"}, axis="columns")

In [ ]:
res.dtypes

In [4]:
table

NameError: name 'table' is not defined

In [8]:
adr.value_counts()

UMCReportId  Adr_Id    MedDRA_Id  Outcome
1            910       10012735   5.0        1
26809723     65993343  10013946   1.0        1
26809736     65993328  10047895   6.0        1
26809738     65993329  10062519   3.0        1
             65993330  10035528   3.0        1
                                            ..
13495371     32580574  10017955   5.0        1
13495375     32580570  10048393   1.0        1
13495376     32580564  10011906   5.0        1
13495377     32580576  10050256   6.0        1
37938522     64184536  10043521   6.0        1
Length: 54840741, dtype: int64